Shishir Poreddy: ECON433: Project 3

Part 1:


Question 1a:

The competitor I chose for Jimmy John's is Subway. I chose Subway as the competitor because of largely well known they are as a sandwhich business. They would be a great benchmark because both business have similar service styles and products, which makes it easier to compare the two. 

I would like to learn the most about the specific states Subway is popular in and compare it to the states that Jimmy John's is popular in. 

NO CODE FOR PART 1.A

Question 1b:

The unit of observation is a single restauarant's location in a single week. 

The Benchmark Long has 104,400 observations and the focal long has 730,800 observations. 

In [2]:
#code 1.b
import pandas as pd
import os

def process_weekly_file(file_path, chain_keyword):
    
    df = pd.read_csv(file_path, compression='gzip')
    
    df_filtered = df[df['brands'].str.contains(chain_keyword, case=False, na=False)].copy()
    
    
    base_name = os.path.basename(file_path)
    year = base_name.split('_')[2]  
    df_filtered['year'] = year
    
    
    
    return df_filtered

weekly_files = [
    "weekly_patterns_2018_sample.csv.gz",
    "weekly_patterns_2019_sample.csv.gz",
    "weekly_patterns_2020_sample.csv.gz",
    "weekly_patterns_2021_sample.csv.gz",
    "weekly_patterns_2022_sample.csv.gz"
]

subway_dfs = []
for file in weekly_files:
    try:
        df_subway = process_weekly_file(file, "Subway")
        subway_dfs.append(df_subway)
    except Exception as e:
        print(f"Error processing {file}: {e}")

benchmark_long = pd.concat(subway_dfs, ignore_index=True)

benchmark_long.to_csv("subway_long_sample.csv", index=False)

print("Benchmark Long (Subway) number of observations:", len(benchmark_long))


focal_long = pd.read_csv("jimmy_johns_long_sample.csv")

print("Focal Long (Jimmy John's) number of observations:", len(focal_long))


/var/folders/fv/b7p7g75s6gx71mfrn6yxcgd00000gn/T/ipykernel_40910/3240002259.py:7: DtypeWarning: Columns (4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, compression='gzip')


Benchmark Long (Subway) number of observations: 104400
Focal Long (Jimmy John's) number of observations: 730800


Question 1c: 

There are 344 unique subway units and 400 unique jimmy johns. 

There 47 unique states for subway and 40 unique states for jimmy johns. 

In [3]:
#code 1.c
import pandas as pd

VALID_US_STATES = {
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY",
    "DC"
}

benchmark_long = pd.read_csv("subway_long_sample.csv")

focal_long = pd.read_csv("jimmy_johns_long_sample.csv")

benchmark_long_us = benchmark_long[benchmark_long['region'].isin(VALID_US_STATES)].copy()
focal_long_us = focal_long[focal_long['region'].isin(VALID_US_STATES)].copy()


num_subway_units = benchmark_long_us['placekey'].nunique()
num_jj_units = focal_long_us['placekey'].nunique()
num_subway_states = benchmark_long_us['region'].nunique()
num_jj_states = focal_long_us['region'].nunique()

print(f"Number of unique Subway units (US only): {num_subway_units}")
print(f"Number of unique Jimmy John's units (US only): {num_jj_units}")
print(f"Number of states Subway operates in (US only): {num_subway_states}")
print(f"Number of states Jimmy John's operates in (US only): {num_jj_states}")



Number of unique Subway units (US only): 344
Number of unique Jimmy John's units (US only): 400
Number of states Subway operates in (US only): 47
Number of states Jimmy John's operates in (US only): 40


Question 2a:

Each row in BenchmarkSummary and FocalSummary is a unique (state, day) combination. 

15,921 observations - Benchmark Summary 
12,006 observations - FocalSummary

The number of observations are different and consistent with my expectations. I think they were different because subway may appear in more states. 

In [4]:
#code 2.a

import ast
import pandas as pd
import numpy as np


def expand_visits_by_day(df, 
                         state_col="region", 
                         start_date_col="date_range_start", 
                         visits_col="visits_by_day", 
                         placekey_col="placekey"):
    
    df[start_date_col] = pd.to_datetime(df[start_date_col], errors="coerce")
    
    df["visits_list"] = df[visits_col].apply(
        lambda x: ast.literal_eval(x) if pd.notnull(x) else []
    )
    
    exploded = df.explode("visits_list").reset_index(drop=True)
    
    exploded["daily_visits"] = exploded["visits_list"].astype(int, errors="ignore")
    
    exploded["day_index"] = exploded.groupby(level=0).cumcount()
    
    exploded["true_date"] = exploded[start_date_col] + pd.to_timedelta(exploded["day_index"], unit="D")
    
    keep_cols = [state_col, placekey_col, "true_date", "daily_visits"]
    for col in [start_date_col]:
        if col in exploded.columns:
            keep_cols.append(col)  
    
    exploded = exploded[keep_cols]
    
    return exploded

def create_summary(df, 
                   state_col="region", 
                   date_col="true_date", 
                   visits_col="daily_visits", 
                   placekey_col="placekey"):
    
    grouped = df.groupby([state_col, date_col])
    summary_df = grouped.agg(
        Nunits=(placekey_col, "nunique"),
        SumDailyVisits=(visits_col, "sum"),
        MaxDailyVisitsPerUnit=(visits_col, "max"),
        MinDailyVisitsPerUnit=(visits_col, "min"),
        MedDailyVisitsPerUnit=(visits_col, "median")
    ).reset_index()
    return summary_df


benchmark_long_expanded = expand_visits_by_day(
    benchmark_long, 
    state_col="region", 
    start_date_col="date_range_start", 
    visits_col="visits_by_day", 
    placekey_col="placekey"
)

focal_long_expanded = expand_visits_by_day(
    focal_long, 
    state_col="region", 
    start_date_col="date_range_start", 
    visits_col="visits_by_day", 
    placekey_col="placekey"
)


BenchmarkSummary = create_summary(
    benchmark_long_expanded,
    state_col="region",
    date_col="true_date",
    visits_col="daily_visits",
    placekey_col="placekey"
)

FocalSummary = create_summary(
    focal_long_expanded,
    state_col="region",
    date_col="true_date",
    visits_col="daily_visits",
    placekey_col="placekey"
)



print("**Unit of Observation**:")
print("Each row in BenchmarkSummary and FocalSummary is a unique (state, day) combination.\n")

print(f"Number of observations in BenchmarkSummary: {len(BenchmarkSummary)}")
print(f"Number of observations in FocalSummary: {len(FocalSummary)}\n")

print("=== BenchmarkSummary (head) ===")
print(BenchmarkSummary.head())

print("\n=== FocalSummary (head) ===")
print(FocalSummary.head())




/var/folders/fv/b7p7g75s6gx71mfrn6yxcgd00000gn/T/ipykernel_40910/2440482986.py:14: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[start_date_col] = pd.to_datetime(df[start_date_col], errors="coerce")
/var/folders/fv/b7p7g75s6gx71mfrn6yxcgd00000gn/T/ipykernel_40910/2440482986.py:26: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  exploded["true_date"] = exploded[start_date_col] + pd.to_timedelta(exploded["day_index"], unit="D")
/Users/shishirporeddy/ECON433/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/shishirporeddy/ECON433/venv/lib/python3.11/

**Unit of Observation**:
Each row in BenchmarkSummary and FocalSummary is a unique (state, day) combination.

Number of observations in BenchmarkSummary: 15921
Number of observations in FocalSummary: 12006

=== BenchmarkSummary (head) ===
  region                  true_date  Nunits SumDailyVisits  \
0     AB  2018-01-01 00:00:00-07:00       8           1043   
1     AB  2018-01-08 00:00:00-07:00       8           1396   
2     AB  2018-01-15 00:00:00-07:00       8           1350   
3     AB  2018-01-22 00:00:00-07:00       8           1458   
4     AB  2018-01-29 00:00:00-07:00       8           1612   

  MaxDailyVisitsPerUnit MinDailyVisitsPerUnit MedDailyVisitsPerUnit  
0                 156.0                   0.0                   1.5  
1                 209.0                   0.0                   1.0  
2                 193.0                   0.0                   2.0  
3                 213.0                   0.0                   2.0  
4                 252.0               

Question 2b:

There are 11223 observations that can be matched between the two datasets. 
The number of observations in Benchmark Summary but not in focalsummary is 4698. 
The number of observations in FocalSummary but not in Benchmark. summary is 783. 

There may imperfect matches because some (state, day) combos may appear for one chain but not the other. Differencyes in store coverages, missing data, or store closures can all cause this. 

In [5]:
#code 2.b

BenchmarkSummary["true_date"] = pd.to_datetime(BenchmarkSummary["true_date"], utc=True, errors='coerce')
FocalSummary["true_date"] = pd.to_datetime(FocalSummary["true_date"], utc=True, errors='coerce')

BenchmarkSummary["true_date"] = BenchmarkSummary["true_date"].dt.tz_localize(None)
FocalSummary["true_date"] = FocalSummary["true_date"].dt.tz_localize(None)

merged_with_indicator = pd.merge(
    BenchmarkSummary, 
    FocalSummary, 
    on=["region", "true_date"], 
    how="outer",
    indicator=True
)
BenchmarkSummary["true_date"] = pd.to_datetime(BenchmarkSummary["true_date"]).dt.tz_localize(None)
FocalSummary["true_date"] = pd.to_datetime(FocalSummary["true_date"]).dt.tz_localize(None)

merged_with_indicator = pd.merge(
    BenchmarkSummary,
    FocalSummary,
    on=["region", "true_date"],
    how="outer",
    indicator=True,
    suffixes=("_benchmark","_focal")
)


import pandas as pd

merged_with_indicator = pd.merge(
    BenchmarkSummary,
    FocalSummary,
    on=["region", "true_date"],  
    how="outer",
    indicator=True,
    suffixes=("_benchmark","_focal")
)

matched = merged_with_indicator[merged_with_indicator["_merge"] == "both"]
only_benchmark = merged_with_indicator[merged_with_indicator["_merge"] == "left_only"]
only_focal = merged_with_indicator[merged_with_indicator["_merge"] == "right_only"]

print("Number of matched observations:", len(matched))
print("Number of observations in BenchmarkSummary but not in FocalSummary:", len(only_benchmark))
print("Number of observations in FocalSummary but not in BenchmarkSummary:", len(only_focal))

MergedSummary = matched.drop(columns=["_merge"])
print("\nFinal MergedSummary (matched only) size:", len(MergedSummary))


Number of matched observations: 11223
Number of observations in BenchmarkSummary but not in FocalSummary: 4698
Number of observations in FocalSummary but not in BenchmarkSummary: 783

Final MergedSummary (matched only) size: 11223


In [6]:
#code 2.b continued
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import ttest_rel


merged_with_indicator = pd.merge(
    BenchmarkSummary,
    FocalSummary,
    on=["region", "true_date"],
    how="outer",
    indicator=True,
    suffixes=("_benchmark","_focal")
)

matched = merged_with_indicator[merged_with_indicator["_merge"] == "both"].drop(columns=["_merge"])
MergedSummary = matched.copy()  

print("Final MergedSummary size (matched only):", len(MergedSummary))
print("Columns in MergedSummary:", MergedSummary.columns.tolist())


Final MergedSummary size (matched only): 11223
Columns in MergedSummary: ['region', 'true_date', 'Nunits_benchmark', 'SumDailyVisits_benchmark', 'MaxDailyVisitsPerUnit_benchmark', 'MinDailyVisitsPerUnit_benchmark', 'MedDailyVisitsPerUnit_benchmark', 'Nunits_focal', 'SumDailyVisits_focal', 'MaxDailyVisitsPerUnit_focal', 'MinDailyVisitsPerUnit_focal', 'MedDailyVisitsPerUnit_focal']


Question 3a:

The average sumdailyvisits for bencmark was 1415.499 and for focal is 6985.399. The focal chain is a lot larger than the benchmark chain, which means that there is more foot traffic per day at the benchmark chain. 

Question 3b:

Since the p-value is well below 0.05, the difference is statistically significant at the 95% confidence level. 

This finding confirms my prior answer in question 3a as focals averages are higher than benchmarks averages. 

In [10]:
#code 3.a
avg_benchmark = MergedSummary["SumDailyVisits_benchmark"].mean()
avg_focal = MergedSummary["SumDailyVisits_focal"].mean()

comparison_table = pd.DataFrame({
    "Chain": ["Benchmark", "Focal"],
    "AvgSumDailyVisits": [avg_benchmark, avg_focal]
})

print("\n=== Question 3a ===")
print("Average SumDailyVisits for the two chains:")
print(comparison_table)

print("\nTakeaway:")
print("Write at least one sentence about how the averages differ and what that implies.")


#code 3.b
import pandas as pd

# coerce to floats
MergedSummary["SumDailyVisits_benchmark"] = pd.to_numeric(
    MergedSummary["SumDailyVisits_benchmark"], errors="coerce"
)
MergedSummary["SumDailyVisits_focal"] = pd.to_numeric(
    MergedSummary["SumDailyVisits_focal"],   errors="coerce"
)

# drop any rows with missing pairs
MergedSummary = MergedSummary.dropna(
    subset=["SumDailyVisits_benchmark","SumDailyVisits_focal"]
)

ttest_result = ttest_rel(
    MergedSummary["SumDailyVisits_benchmark"],
    MergedSummary["SumDailyVisits_focal"]
)

print("\n=== Question 3b ===")
print("Paired T-test results for difference in SumDailyVisits (Benchmark vs. Focal):")
print(f"  t-statistic: {ttest_result.statistic}")
print(f"  p-value:     {ttest_result.pvalue}")

alpha = 0.05
if ttest_result.pvalue < alpha:
    print("Conclusion: The difference is statistically significant at 95% confidence.")
else:
    print("Conclusion: The difference is NOT statistically significant at 95% confidence.")





=== Question 3a ===
Average SumDailyVisits for the two chains:
       Chain  AvgSumDailyVisits
0  Benchmark        1415.498975
1      Focal        6985.399358

Takeaway:
Write at least one sentence about how the averages differ and what that implies.

=== Question 3b ===
Paired T-test results for difference in SumDailyVisits (Benchmark vs. Focal):
  t-statistic: -25.647633618564214
  p-value:     4.94990030503241e-141
Conclusion: The difference is statistically significant at 95% confidence.


In [11]:
print(MergedSummary["SumDailyVisits_focal"].dtype)
print(MergedSummary["SumDailyVisits_focal"].head(10)) 


print(MergedSummary["SumDailyVisits_focal"].apply(type).value_counts())


int64
261    1008
262    1134
263     882
264     616
265     707
266     784
267     819
268     665
269     763
270    1638
Name: SumDailyVisits_focal, dtype: int64
SumDailyVisits_focal
<class 'int'>    11223
Name: count, dtype: int64


In [12]:
type_counts = MergedSummary["SumDailyVisits_focal"].apply(type).value_counts()
print(type_counts)
MergedSummary["SumDailyVisits_focal"] = pd.to_numeric(
    MergedSummary["SumDailyVisits_focal"], 
    errors="coerce"
)
MergedSummary.dropna(subset=["SumDailyVisits_focal"], inplace=True)

MergedSummary["ln_SumDailyVisits_focal"] = np.log(MergedSummary["SumDailyVisits_focal"] + 1)


SumDailyVisits_focal
<class 'int'>    11223
Name: count, dtype: int64


Question 4a:

BlueState: The coefficient is positive and significant. This means that before march 13, 2020 the average visits in blue states were higher than in red states. They receive an economic advantage in blue states. 

Post0313202: The coeffienct is negative and not significant. This means that we can not come to a conclusion that average visits changed by a significant amount in red states post march 13th. There is no strong evidence for economical benefits of this variable. 

Blue_Post: The coefficnet is negative not significant. The p-value was not below the 5%. threshold, therefore we can't assume there is any differential effects at the levels of significance. 

Fail to reject the null hypothesis at the 5% siginificance level that the national emergency data made no difference in the average visits between red and blue states. The bordeline p-value suggests there may be an effect, but at the 95% confidence level there was no definitive answer, resulting in an inconclusive claim that a shift occured post march 13, 2020. 

Question 4b:

(1)The state fixed effects remove time invarient differences across states. The BlueState coefficient is largely absorbed by these dummies, making it less meaningful on its own. 

(2)The date fixed effects remove day-to-day shocks that affects all states, such as national events or holidays. This would cause the post variable to be overshadowed by daily dummies. 

(3)The use of both state and date fixed effects would allow for control for all state specific and day specific unobserved factors. 

I would prefer the state and date fixed effects because it effective in testing how different types of groups change once a poicly had been updated. This way it allows to net out both state level and day confounders. 


Question 4c:

This would effect my findings in part a and b since the differences in logs is focused on the gap between the chains visits and the benchmarks visits isntead of their individual levels. This would allow for any common effect that hits both the chains at the same time to be canceled out. 

In [13]:
#code 4.a
print("\n=== Question 4a ===")

blue_states_2016 = {
    "CA", "NY", "MA", "WA", "OR", "NJ", "IL", "HI", "MD", "VT", "RI",
    "CT", "CO", "VA", "NM", "NH", "ME", "NV", "MN"  
}

MergedSummary["true_date"] = pd.to_datetime(MergedSummary["true_date"], errors="coerce")

MergedSummary["BlueState"] = MergedSummary["region"].isin(blue_states_2016).astype(int)
MergedSummary["Post03132020"] = (MergedSummary["true_date"] >= "2020-03-13").astype(int)
MergedSummary["Blue_Post"] = MergedSummary["BlueState"] * MergedSummary["Post03132020"]

MergedSummary["ln_SumDailyVisits_focal"] = np.log(MergedSummary["SumDailyVisits_focal"] + 1)

X = MergedSummary[["BlueState","Post03132020","Blue_Post"]]
X = sm.add_constant(X)
y = MergedSummary["ln_SumDailyVisits_focal"]

model = sm.OLS(y, X, missing='drop').fit()
print(model.summary())

print("\nInterpretation of coefficients:")
print("- BlueState: Baseline difference between blue vs. red states (before 3/13/2020).")
print("- Post03132020: Shift after March 13, 2020 in red states.")
print("- Blue_Post: Additional shift for blue states post-3/13/2020.")
print("Check significance (p-values < 0.05) and interpret the economic meaning.")
print("Then address the null hypothesis about the national emergency date.")

import pandas as pd
import numpy as np
import statsmodels.api as sm



#code 4.b
print("\n=== Question 4b ===")
print("How would results change with state FE, date FE, or both?")

X = MergedSummary[["BlueState", "Post03132020", "Blue_Post"]]
X = sm.add_constant(X)  
y = MergedSummary["ln_SumDailyVisits_focal"]

state_dummies = pd.get_dummies(MergedSummary["region"], drop_first=True)
X_state_fe = pd.concat([X, state_dummies], axis=1).astype(float)
model_state_fe = sm.OLS(y, X_state_fe, missing='drop').fit()

MergedSummary["true_date_str"] = MergedSummary["true_date"].astype(str)
date_dummies = pd.get_dummies(MergedSummary["true_date_str"], drop_first=True)
X_date_fe = pd.concat([X, date_dummies], axis=1).astype(float)
model_date_fe = sm.OLS(y, X_date_fe, missing='drop').fit()

X_both_fe = pd.concat([X, state_dummies, date_dummies], axis=1).astype(float)
model_both_fe = sm.OLS(y, X_both_fe, missing='drop').fit()

print("=== Model with State FE ===")
print(model_state_fe.summary())
print("\n=== Model with Date FE (using string conversion) ===")
print(model_date_fe.summary())
print("\n=== Model with Both State & Date FE ===")
print(model_both_fe.summary())

#code 4.c

MergedSummary["SumDailyVisits_benchmark"] = pd.to_numeric(
    MergedSummary["SumDailyVisits_benchmark"],
    errors="coerce"
)

MergedSummary.dropna(subset=["SumDailyVisits_benchmark"], inplace=True)

MergedSummary["SumDailyVisits_benchmark"] = MergedSummary["SumDailyVisits_benchmark"].clip(lower=0)

MergedSummary["ln_SumDailyVisits_benchmark"] = np.log(
    MergedSummary["SumDailyVisits_benchmark"] + 1
)

MergedSummary["dep_var_diff"] = (
    MergedSummary["ln_SumDailyVisits_focal"] - MergedSummary["ln_SumDailyVisits_benchmark"]
)

X_diff = MergedSummary[["BlueState","Post03132020","Blue_Post"]]
X_diff = sm.add_constant(X_diff)
y_diff = MergedSummary["dep_var_diff"]

model_diff = sm.OLS(y_diff, X_diff, missing='drop').fit()
print(model_diff.summary())





=== Question 4a ===
                               OLS Regression Results                              
Dep. Variable:     ln_SumDailyVisits_focal   R-squared:                       0.007
Model:                                 OLS   Adj. R-squared:                  0.007
Method:                      Least Squares   F-statistic:                     26.47
Date:                     Thu, 17 Apr 2025   Prob (F-statistic):           4.74e-17
Time:                             22:57:53   Log-Likelihood:                -23703.
No. Observations:                    11223   AIC:                         4.741e+04
Df Residuals:                        11219   BIC:                         4.744e+04
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------